### Запрос: рассказать об этапаx работы с курсовым - как шло улучшение.
    При этом есть проблема - я затирал код, т.к. это соревнование и нужен
    только вектор прогнозов
    
__Общий подход__:
- так как был почти месяц для работы и каждый день было по 10 возможных попыток, не заморачиваться над красотой кода, делать топорно, быстро проверяя гипотезы.
- оставить тюнинг параметров в конец. Используем просто HO валидацию, сделав один раз подбор параметров через StratifiedKFold CV, 5 фолдов, RandomSearch
- раз в неделю повторяя поиск на всякий случай, но он был достаточно стабильный: для CatBoost n_estimators = 1500 глубина 3 или 4.

__Этапы работы__:

1) как и все сделал с таблицей клиентских профилей, используя Catboost из коробки,получил как и все порядка 0,71.

2) сразу приступил к сращиванию сета с другими таблицами, начиная от одной, и кончая всеми
   - разбил все таблицы на 2 сета по признаку Cash or Credit card, потом уже группировал отдельно, сращивал эти 2 таблицы и добавлял в трейн и тест. Получалось немного лучше, чем простое группирование, без разбиения на Cash and Credit card
   - сначала группировал, оставляя первое или последнее вхождение
   - потом группировал, как предложил Никита: max, min, mean, ...
        Получалось лучше при min.
   сделал много попыток ~ 20, результат ... ноль :)

3) вернулся к таблице клиентского профиля и сделал EDA.
   Выводы:
   
    Выбросы:
   - TOTAL_SALARY есть выброс, заменям на median
   - DAYS_ON_LAST_JOB есть выброс, заменяем на nan
   - OWN_CAR_AGE есть выброс (>=64), заменяем на nan
   
    Объединение категорий в дискретных признаках:
   - CHILDRENS - можно объединить группы с 0, 1 и 2 ребёнка
   - FAMILY_SIZE - можно объединить группы с 1, 2, 3 и 4 человека
   - AMT_REQ_CREDIT_BUREAU_YEAR -  можно объединить группы с 0 до 7,включительно
   
    Распределение фич по цели в тесте и трейне - одинаковое, сюрпризов не будет
   
    Важность фич:
   - корреляция -  "FLAG_EMAIL" и "AMT_REQ_CREDIT_BUREAU_DAY" можно сразу отбросить
   - методика WOE/IV - даёт схожие результаты
   - использования слабой модели RandomForest даёт идеи отбросить:
     'AMT_REQ_CREDIT_BUREAU_HOUR','AMT_REQ_CREDIT_BUREAU_WEEK','FAMILY_SIZE' и 'APPLICATION_NUMBER', конечно
    
    Балансировка классов - результатов не дала (
    
    Довольно долго сидел на ловле макимума от удаления каждой слабой фичи (RandomForest фича имп), в конце на catb
    
    Вернулся к другим таблицам - получал вектора с каждой таблицы, добавлял, удалял - нет результатов, после анализа причин - очень много пропусков после агрегации
    
4) Генерация новых фич (перед этим этапом я подошёл уже близко к Baseline) 
- тут Никита вне конкуренции. Мои апплодисменты) Сколько я не пытался что-то модернизировать, добавить что-то своё, ничего не улучшало, в лучшем случае почти не ухудшало, а то и здорово ухудшало. 
- Кодирование категориальны фич - сделал высовым, через value_counts

5) Ансамблирование.
- ввёл LGBM, мои параметры были: дропнул категории, {'max_depth': 3, 'n_estimators': 300}
- подобрал лучшую комбинацию, у меня была такая: 

X_test_f['TARGET_F'] = np.sqrt(X_test_['TARGET'] * X_test__['TARGET'])

__Всё, получил что получил, спасибо за внимание)__


In [1]:
from typing import List, Optional
from tqdm import tqdm
import time

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns
import scipy.stats as st
from scipy.stats import probplot, ks_2samp, shapiro, ttest_ind, mannwhitneyu
from scipy.stats import chi2_contingency, ttest_rel, kurtosis
from statsmodels.stats.weightstats import zconfint

# импортирование библиотек моделей
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb, lightgbm as lgbm, catboost as catb

from sklearn.metrics import roc_auc_score, r2_score, classification_report, make_scorer
from sklearn.model_selection import KFold, cross_val_score, train_test_split, ShuffleSplit
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, learning_curve, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted
import missingno as msno
%matplotlib inline

# убираем лишние замечания интерпретатора
import warnings
warnings.simplefilter('ignore')

# import additional packages for WOE/IV
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string
# глобальные переменные WOE/IV
max_bin = 20
force_bin = 3

import gc
pd.options.display.max_columns = 50

In [2]:
def get_classification_report(y_train_true, y_train_pred, y_test_true, y_test_pred):
    print('TRAIN\n\n' + classification_report(y_train_true, y_train_pred))
    print('TEST\n\n' + classification_report(y_test_true, y_test_pred))
    print('CONFUSION MATRIX\n')
    print(pd.crosstab(y_test_true, y_test_pred))

In [3]:
# dataset loading
train = pd.read_csv("/home/pi/GeekBrains/Kaggle/competition/train.csv")
test = pd.read_csv("/home/pi/GeekBrains/Kaggle/competition/test.csv")
client_profile = pd.read_csv("/home/pi/GeekBrains/Kaggle/competition/client_profile.csv")
bki = pd.read_csv("/home/pi/GeekBrains/Kaggle/competition/bki.csv")
payments = pd.read_csv("/home/pi/GeekBrains/Kaggle/competition/payments.csv")
applications = pd.read_csv("/home/pi/GeekBrains/Kaggle/competition/applications_history.csv")

print("train.shape = {} rows, {} cols".format(*train.shape))
print("test.shape = {} rows, {} cols".format(*test.shape))
print("client_profile.shape = {} rows, {} cols".format(*client_profile.shape))
print("bki.shape = {} rows, {} cols".format(*bki.shape))
print("payments.shape = {} rows, {} cols".format(*payments.shape))
print("applications.shape = {} rows, {} cols".format(*applications.shape))

train.shape = 110093 rows, 3 cols
test.shape = 165141 rows, 2 cols
client_profile.shape = 250000 rows, 24 cols
bki.shape = 945234 rows, 17 cols
payments.shape = 1023932 rows, 8 cols
applications.shape = 1670214 rows, 26 cols


In [4]:
X_train = train.copy()
X_test = test.copy()
X_client_profile = client_profile.copy()
X_bki = bki.copy()
X_payments = payments.copy()
X_applications = applications.copy()

X_train = X_train.sort_values(by = 'APPLICATION_NUMBER')
X_train = X_train.reset_index(drop=True)
X_bki = X_bki.sort_values(by = 'APPLICATION_NUMBER')
X_bki = X_bki.reset_index(drop=True)
X_client_profile = X_client_profile.sort_values(by = 'APPLICATION_NUMBER')
X_client_profile = X_client_profile.reset_index(drop=True)
X_payments = X_payments.sort_values(by = 'APPLICATION_NUMBER')
X_payments = X_payments.reset_index(drop=True)
X_applications = X_applications.sort_values(by = 'APPLICATION_NUMBER')
X_applications = X_applications.reset_index(drop=True)
X_test = X_test.sort_values(by = 'APPLICATION_NUMBER')
X_test = X_test.reset_index(drop=True)

print("APPLICATION_NUMBER_train = {} ".format(X_train['APPLICATION_NUMBER'].nunique()))
print("APPLICATION_NUMBER_test = {} ".format(X_test['APPLICATION_NUMBER'].nunique()))
print("APPLICATION_NUMBER_client_profile = {} ".format(X_client_profile['APPLICATION_NUMBER'].nunique()))
print("APPLICATION_NUMBER_bki = {} ".format(X_bki['APPLICATION_NUMBER'].nunique()))
print("APPLICATION_NUMBER_payments = {} ".format(X_payments['APPLICATION_NUMBER'].nunique()))
print("APPLICATION_NUMBER_applications = {} ".format(X_applications['APPLICATION_NUMBER'].nunique()))

APPLICATION_NUMBER_train = 110093 
APPLICATION_NUMBER_test = 165141 
APPLICATION_NUMBER_client_profile = 250000 
APPLICATION_NUMBER_bki = 273131 
APPLICATION_NUMBER_payments = 264726 
APPLICATION_NUMBER_applications = 338857 


In [5]:
def create_profile_features(X: pd.DataFrame, copy: bool = True) -> pd.DataFrame:
    """
    Создание признаков на основе профиля клиентов.

    Parameters
    ----------
    X: pandas.core.frame.DataFrame
        Матрица признаков с исходным профилем клиента.

    copy: bool, optional, default = True
        Флаг использования копии датафрейма X.
        Опциональный параметр, по умолчанию, равен True.

    Returns
    -------
    X: pandas.core.frame.DataFrame
        Расширенная матрица признаков с профилем клиентов.

    """
    if copy:
        X = X.copy()

    X["EXT_SCOR_PROD"] = X["EXTERNAL_SCORING_RATING_1"] * X["EXTERNAL_SCORING_RATING_2"] * X["EXTERNAL_SCORING_RATING_3"]
    X["EXT_SCOR_EIGHTED"] = X.EXTERNAL_SCORING_RATING_1 * 2 + X.EXTERNAL_SCORING_RATING_2 * 1 + X.EXTERNAL_SCORING_RATING_3 * 3

    for function_name in ["min", "max", "mean", "nanmedian", "var"]:
        feature_name = "external_scoring_rating_{}".format(function_name)
        X[feature_name] = eval("np.{}".format(function_name))(
            X[["EXTERNAL_SCORING_RATING_1", "EXTERNAL_SCORING_RATING_2", "EXTERNAL_SCORING_RATING_3"]], axis=1
        )

    # Отношение между основными фин. показателями
    X['ratio_credit_to_annuity'] = X['AMOUNT_CREDIT'] / X['AMOUNT_ANNUITY']
    X["ratio_annuity_to_salary"] = X['AMOUNT_ANNUITY'] / X['TOTAL_SALARY']
    X['ratio_credit_to_salary'] = X['AMOUNT_CREDIT'] / X['TOTAL_SALARY']
    #X["total_salary_net"] = X["total_salary"] - X["amount_annuity"]

    # Отношение фин. показателей к возрасту и временным фичам
    X["ratio_annuity_to_age"] = X["AMOUNT_ANNUITY"] / X["AGE"]
    X["ratio_credit_to_age"] = X["AMOUNT_CREDIT"] / X["AGE"]
    X["ratio_salary_to_age"] = X["TOTAL_SALARY"] / X["AGE"]
    X["ratio_salary_to_experience"] = X["TOTAL_SALARY"] / X["DAYS_ON_LAST_JOB"]
    X["ratio_credit_to_experience"] = X["AMOUNT_CREDIT"] / X["DAYS_ON_LAST_JOB"]
    X["ratio_annuity_to_experience"] = X["AMOUNT_ANNUITY"] / X["DAYS_ON_LAST_JOB"]

    # Отношение врменных признаков
    X["ratio_age_to_experience"] = X["AGE"] / X["DAYS_ON_LAST_JOB"]
    X["ratio_salary_to_region_population"] = X["TOTAL_SALARY"] * X["REGION_POPULATION"]
    X["ratio_car_to_experience"] = X["OWN_CAR_AGE"] / X["DAYS_ON_LAST_JOB"]
    X["ratio_car_to_age"] = X["OWN_CAR_AGE"] / X["AGE"]

    # Произведение фин. показателей кредита на вероятность дефолта
    # Такая штука называется математическим ожиданием дефолта или ожидаемыми потерями
    X["expected_total_loss_1"] = X["EXTERNAL_SCORING_RATING_1"] * X["AMOUNT_CREDIT"]
    X["expected_total_loss_2"] = X["EXTERNAL_SCORING_RATING_2"] * X["AMOUNT_CREDIT"]
    X["expected_total_loss_3"] = X["EXTERNAL_SCORING_RATING_3"] * X["AMOUNT_CREDIT"]
    X["expected_monthly_loss_1"] = X["EXTERNAL_SCORING_RATING_1"] * X["AMOUNT_ANNUITY"]
    X["expected_monthly_loss_2"] = X["EXTERNAL_SCORING_RATING_2"] * X["AMOUNT_ANNUITY"]
    X["expected_monthly_loss_3"] = X["EXTERNAL_SCORING_RATING_3"] * X["AMOUNT_ANNUITY"]

    return X

In [6]:
X_client_profile_ = create_profile_features(X_client_profile)
X_client_profile_.head(n=2)

APPLICATION_NUMBER GENDER  CHILDRENS  TOTAL_SALARY  AMOUNT_CREDIT  \
0           123423341      M          0      202500.0       406597.5   
1           123423342      F          0      270000.0      1293502.5   

   AMOUNT_ANNUITY                EDUCATION_LEVEL         FAMILY_STATUS  \
0         24700.5  Secondary / secondary special  Single / not married   
1         35698.5               Higher education               Married   

   REGION_POPULATION    AGE  DAYS_ON_LAST_JOB  OWN_CAR_AGE  FLAG_PHONE  \
0           0.018801   9461               637          NaN           1   
1           0.003541  16765              1188          NaN           1   

   FLAG_EMAIL  FAMILY_SIZE  EXTERNAL_SCORING_RATING_1  \
0           0          1.0                   0.083037   
1           0          2.0                   0.311267   

   EXTERNAL_SCORING_RATING_2  EXTERNAL_SCORING_RATING_3  \
0                   0.262949                   0.139376   
1                   0.622246                        NaN   

   AMT_REQ_CREDIT_BUREAU_HOUR  AMT_REQ_CREDIT_BUREAU_DAY  \
0                         0.0                        0.0   
1                         0.0                        0.0   

   AMT_REQ_CREDIT_BUREAU_WEEK  AMT_REQ_CREDIT_BUREAU_MON  \
0                         0.0                        0.0   
1                         0.0                        0.0   

   AMT_REQ_CREDIT_BUREAU_QRT  AMT_REQ_CREDIT_BUREAU_YEAR  EXT_SCOR_PROD  \
0                        0.0                         1.0       0.003043   
1                        0.0                         0.0            NaN   

   EXT_SCOR_EIGHTED  external_scoring_rating_min  external_scoring_rating_max  \
0           0.84715                     0.083037                     0.262949   
1               NaN                     0.311267                     0.622246   

   external_scoring_rating_mean  external_scoring_rating_nanmedian  \
0                      0.161787                           0.139376   
1                      0.466757                           0.466757   

   external_scoring_rating_var  ratio_credit_to_annuity  \
0                     0.005646                16.461104   
1                     0.024177                36.234085   

   ratio_annuity_to_salary  ratio_credit_to_salary  ratio_annuity_to_age  \
0                 0.121978                2.007889              2.610771   
1                 0.132217                4.790750              2.129347   

   ratio_credit_to_age  ratio_salary_to_age  ratio_salary_to_experience  \
0            42.976165            21.403657                  317.896389   
1            77.154936            16.104981                  227.272727   

   ratio_credit_to_experience  ratio_annuity_to_experience  \
0                  638.300628                    38.776295   
1                 1088.806818                    30.049242   

   ratio_age_to_experience  ratio_salary_to_region_population  \
0                14.852433                          3807.2025   
1                14.111953                           956.0700   

   ratio_car_to_experience  ratio_car_to_age  expected_total_loss_1  \
0                      NaN               NaN           33762.623349   
1                      NaN               NaN          402625.045440   

   expected_total_loss_2  expected_total_loss_3  expected_monthly_loss_1  \
0          106914.240440           56669.843749              2051.054613   
1          804876.465907                    NaN             11111.776115   

   expected_monthly_loss_2  expected_monthly_loss_3  
0              6494.961715              3442.651456  
1             22213.240808                      NaN

In [7]:
X_train = pd.merge(X_train, X_client_profile_, how='outer')
X_train = X_train.sort_values(by = ['APPLICATION_NUMBER'])
X_train = X_train.reset_index(drop=True)
X_test = pd.merge(X_test, X_client_profile_, how='outer')
X_test = X_test.sort_values(by = ['APPLICATION_NUMBER'])
X_test = X_test.reset_index(drop=True)
X_train.shape

(270554, 52)

In [8]:
X_train.loc[X_train['GENDER'].isna(), 'GENDER'] = 'zero'
X_test.loc[X_test['GENDER'].isna(), 'GENDER'] = 'zero'
X_train.loc[X_train['EDUCATION_LEVEL'].isna(), 'EDUCATION_LEVEL'] = 'zero'
X_test.loc[X_test['EDUCATION_LEVEL'].isna(), 'EDUCATION_LEVEL'] = 'zero'
X_train.loc[X_train['FAMILY_STATUS'].isna(), 'FAMILY_STATUS'] = 'zero'
X_test.loc[X_test['FAMILY_STATUS'].isna(), 'FAMILY_STATUS'] = 'zero'
X_train.shape

(270554, 52)

In [9]:
freq_encoder = X_train["GENDER"].value_counts(normalize=True)
X_train["GENDER_FREQ_ENC"] = X_train["GENDER"].map(freq_encoder)
freq_encoder = X_train["EDUCATION_LEVEL"].value_counts(normalize=True)
X_train["EDUCATION_LEVEL_FREQ_ENC"] = X_train["EDUCATION_LEVEL"].map(freq_encoder)
freq_encoder = X_train["FAMILY_STATUS"].value_counts(normalize=True)
X_train["FAMILY_STATUS_FREQ_ENC"] = X_train["FAMILY_STATUS"].map(freq_encoder)
freq_encoder = X_train["NAME_CONTRACT_TYPE"].value_counts(normalize=True)
X_train["NAME_CONTRACT_TYPE_FREQ_ENC"] = X_train["NAME_CONTRACT_TYPE"].map(freq_encoder)

freq_encoder = X_test["GENDER"].value_counts(normalize=True)
X_test["GENDER_FREQ_ENC"] = X_test["GENDER"].map(freq_encoder)
freq_encoder = X_test["EDUCATION_LEVEL"].value_counts(normalize=True)
X_test["EDUCATION_LEVEL_FREQ_ENC"] = X_test["EDUCATION_LEVEL"].map(freq_encoder)
freq_encoder = X_test["FAMILY_STATUS"].value_counts(normalize=True)
X_test["FAMILY_STATUS_FREQ_ENC"] = X_test["FAMILY_STATUS"].map(freq_encoder)
freq_encoder = X_test["NAME_CONTRACT_TYPE"].value_counts(normalize=True)
X_test["NAME_CONTRACT_TYPE_FREQ_ENC"] = X_test["NAME_CONTRACT_TYPE"].map(freq_encoder)

In [10]:
X_train = X_train.loc[~X_train['NAME_CONTRACT_TYPE'].isna()]
X_test = X_test.loc[~X_test['NAME_CONTRACT_TYPE'].isna()]

y_train = X_train['TARGET']
X_train = X_train.drop(['TARGET'], axis=1)
X_train.shape

(110093, 55)

In [11]:
X_test.loc[X_test['TOTAL_SALARY'] > 100000000, 'TOTAL_SALARY'] = X_test['TOTAL_SALARY'].median()
X_test.loc[X_test['DAYS_ON_LAST_JOB'] > 360000, 'DAYS_ON_LAST_JOB'] = np.nan
X_train.loc[X_train['DAYS_ON_LAST_JOB'] > 360000, 'DAYS_ON_LAST_JOB'] = np.nan
X_train.loc[X_train['OWN_CAR_AGE'] >= 64, 'OWN_CAR_AGE'] = np.nan
X_test.loc[X_test['OWN_CAR_AGE'] >= 64, 'OWN_CAR_AGE'] = np.nan

In [12]:
X_train.loc[(X_train['CHILDRENS'] == 0) | (X_train['CHILDRENS'] == 1) | (X_train['CHILDRENS'] == 2), 'CHILDRENS'] = 1
X_train.loc[(X_train['FAMILY_SIZE'] == 4) | (X_train['FAMILY_SIZE'] == 1) | (X_train['FAMILY_SIZE'] == 2) | (X_train['FAMILY_SIZE'] == 3), 'FAMILY_SIZE'] = 1
X_train.loc[(X_train['AMT_REQ_CREDIT_BUREAU_YEAR'] == 0) | (X_train['AMT_REQ_CREDIT_BUREAU_YEAR'] == 4) | (X_train['AMT_REQ_CREDIT_BUREAU_YEAR'] == 1) | (X_train['AMT_REQ_CREDIT_BUREAU_YEAR'] == 2) | (X_train['AMT_REQ_CREDIT_BUREAU_YEAR'] == 3) | (X_train['AMT_REQ_CREDIT_BUREAU_YEAR'] == 5) | (X_train['AMT_REQ_CREDIT_BUREAU_YEAR'] == 6) | (X_train['AMT_REQ_CREDIT_BUREAU_YEAR'] == 7), 'AMT_REQ_CREDIT_BUREAU_YEAR'] = 1

X_test.loc[(X_test['CHILDRENS'] == 0) | (X_test['CHILDRENS'] == 1) | (X_test['CHILDRENS'] == 2), 'CHILDRENS'] = 1
X_test.loc[(X_test['FAMILY_SIZE'] == 4) | (X_test['FAMILY_SIZE'] == 1) | (X_test['FAMILY_SIZE'] == 2) | (X_test['FAMILY_SIZE'] == 3), 'FAMILY_SIZE'] = 1
X_test.loc[(X_test['AMT_REQ_CREDIT_BUREAU_YEAR'] == 0) | (X_test['AMT_REQ_CREDIT_BUREAU_YEAR'] == 4) | (X_test['AMT_REQ_CREDIT_BUREAU_YEAR'] == 1) | (X_test['AMT_REQ_CREDIT_BUREAU_YEAR'] == 2) | (X_test['AMT_REQ_CREDIT_BUREAU_YEAR'] == 3) | (X_test['AMT_REQ_CREDIT_BUREAU_YEAR'] == 5) | (X_test['AMT_REQ_CREDIT_BUREAU_YEAR'] == 6) | (X_test['AMT_REQ_CREDIT_BUREAU_YEAR'] == 7), 'AMT_REQ_CREDIT_BUREAU_YEAR'] = 1

In [13]:
X_train = X_train.drop(['FLAG_EMAIL','AMT_REQ_CREDIT_BUREAU_HOUR',
                        'AMT_REQ_CREDIT_BUREAU_WEEK','FAMILY_SIZE',
                        'AMT_REQ_CREDIT_BUREAU_DAY'], axis=1)
X_test = X_test.drop(['FLAG_EMAIL','AMT_REQ_CREDIT_BUREAU_HOUR',
                        'AMT_REQ_CREDIT_BUREAU_WEEK','FAMILY_SIZE',
                        'AMT_REQ_CREDIT_BUREAU_DAY'], axis=1)

In [14]:
X_train = X_train.drop(['APPLICATION_NUMBER'], axis=1)
X_test = X_test.drop(['APPLICATION_NUMBER'], axis=1)

X_trainn, X_testt, y_trainn, y_testt = train_test_split(X_train, y_train, shuffle=True, test_size=0.3, random_state=21)

In [15]:
X_trainn, X_testt, y_trainn, y_testt = train_test_split(X_train, y_train, shuffle=True, test_size=0.3, random_state=21)

In [ ]:
model_catb = catb.CatBoostClassifier(cat_features=['NAME_CONTRACT_TYPE', 'GENDER',
                                                   'EDUCATION_LEVEL',
                                                   'FAMILY_STATUS'],
                                     silent=True, random_state=21)
model_catb.fit(X_trainn, y_trainn)

y_train_pred = model_catb.predict(X_trainn)
y_test_pred = model_catb.predict(X_testt)

get_classification_report(y_trainn, y_train_pred, y_testt, y_test_pred)

In [ ]:
print(roc_auc_score(y_trainn, model_catb.predict_proba(X_trainn)[:,1]))
print(roc_auc_score(y_testt, model_catb.predict_proba(X_testt)[:,1]))

In [ ]:
model_catb.fit(X_train, y_train)

In [ ]:
TARGET = pd.DataFrame(model_catb.predict_proba(X_test))
TARGET =TARGET.rename(columns={0:0, 1:'TARGET'}).drop(0, axis=1)
TARGET.head()

In [ ]:
X_test_ = test.copy()
X_test_ = X_test_.sort_values(by = 'APPLICATION_NUMBER')
X_test_ = X_test_.reset_index(drop=True)
X_test_['TARGET'] = TARGET
X_test_ = X_test_.drop(['NAME_CONTRACT_TYPE'], axis=1)
X_test_.head()

In [ ]:
X_test_.to_csv('test_submission.csv', index=False)

### Выбор лучшей модели и подбор гиперпараметров<a class="anchor" id="tuning_best_model"></a> 

In [ ]:
categorical = ['NAME_CONTRACT_TYPE','GENDER','EDUCATION_LEVEL','FAMILY_STATUS']
model_catb = catb.CatBoostClassifier(cat_features=categorical, silent=True, random_state=21)

In [ ]:
params = {'n_estimators':[50, 100, 500, 1000, 1500],
          'max_depth':[3, 5, 7]}

In [ ]:
cv=StratifiedKFold(n_splits=5, random_state=21, shuffle=True)

In [ ]:
%%time

rs = RandomizedSearchCV(model_catb, params, scoring='roc_auc', cv=cv, n_jobs=-1)
rs.fit(X_train, y_train)

In [ ]:
rs.best_params_

In [ ]:
rs.best_score_

In [16]:
%%time

final_model = catb.CatBoostClassifier(cat_features=['NAME_CONTRACT_TYPE', 'GENDER',
                                                   'EDUCATION_LEVEL','FAMILY_STATUS'],
                                      n_estimators=1500, max_depth=4, silent=True,
                                      random_state=21)
final_model.fit(X_trainn, y_trainn)

y_train_pred = final_model.predict(X_trainn)
y_test_pred = final_model.predict(X_testt)

get_classification_report(y_trainn, y_train_pred, y_testt, y_test_pred)

TRAIN

              precision    recall  f1-score   support

         0.0       0.92      1.00      0.96     70839
         1.0       0.93      0.05      0.09      6226

    accuracy                           0.92     77065
   macro avg       0.93      0.52      0.53     77065
weighted avg       0.92      0.92      0.89     77065

TEST

              precision    recall  f1-score   support

         0.0       0.92      1.00      0.96     30357
         1.0       0.53      0.02      0.04      2671

    accuracy                           0.92     33028
   macro avg       0.73      0.51      0.50     33028
weighted avg       0.89      0.92      0.88     33028

CONFUSION MATRIX

col_0     0.0  1.0
TARGET            
0.0     30309   48
1.0      2616   55
CPU times: user 4min 14s, sys: 16.5 s, total: 4min 31s
Wall time: 1min 15s


In [17]:
print(roc_auc_score(y_trainn, final_model.predict_proba(X_trainn)[:,1]))
print(roc_auc_score(y_testt, final_model.predict_proba(X_testt)[:,1]))

0.7848751744991822
0.7297215919772233


In [18]:
final_model.fit(X_train, y_train)

In [19]:
TARGET = pd.DataFrame(final_model.predict_proba(X_test))
TARGET =TARGET.rename(columns={0:0, 1:'TARGET'}).drop(0, axis=1)
TARGET.head()

TARGET
0  0.054287
1  0.040208
2  0.045050
3  0.075531
4  0.004340

In [20]:
X_test_ = test.copy()
X_test_ = X_test_.sort_values(by = 'APPLICATION_NUMBER')
X_test_ = X_test_.reset_index(drop=True)
X_test_['TARGET'] = TARGET
X_test_ = X_test_.drop(['NAME_CONTRACT_TYPE'], axis=1)
X_test_.head()

APPLICATION_NUMBER    TARGET
0           123423342  0.054287
1           123423343  0.040208
2           123423344  0.045050
3           123423346  0.075531
4           123423347  0.004340

### LGBM

In [21]:
X_train = X_train.drop(['NAME_CONTRACT_TYPE', 'GENDER',
                        'EDUCATION_LEVEL','FAMILY_STATUS'], axis=1)
X_test = X_test.drop(['NAME_CONTRACT_TYPE', 'GENDER',
                     'EDUCATION_LEVEL','FAMILY_STATUS'], axis=1)

In [62]:
lgb_model = lgbm.LGBMClassifier(random_state=21)

In [63]:
params = {'n_estimators':[300,500,800],
          'max_depth':[2,3,4]}

In [64]:
cv=StratifiedKFold(n_splits=5, random_state=21, shuffle=True)

In [65]:
%%time

gs = GridSearchCV(lgb_model, params, scoring='roc_auc', cv=cv, n_jobs=-1)
gs.fit(X_train, y_train)

CPU times: user 13.4 s, sys: 285 ms, total: 13.7 s
Wall time: 3min 6s


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=21, shuffle=True),
             estimator=LGBMClassifier(random_state=21), n_jobs=-1,
             param_grid={'max_depth': [2, 3, 4],
                         'n_estimators': [300, 500, 800]},
             scoring='roc_auc')

In [66]:
gs.best_params_

{'max_depth': 3, 'n_estimators': 300}

In [68]:
gs.best_score_

0.7288302451612216

In [127]:
lgb_model = lgbm.LGBMClassifier(n_estimators=500, max_depth=3, reg_lambda=0.075, random_state=21)

lgb_model.fit(X_trainn, y_trainn)

y_train_pred = lgb_model.predict(X_trainn)
y_test_pred = lgb_model.predict(X_testt)

get_classification_report(y_trainn, y_train_pred, y_testt, y_test_pred)

TRAIN

              precision    recall  f1-score   support

         0.0       0.92      1.00      0.96     70839
         1.0       0.79      0.03      0.06      6226

    accuracy                           0.92     77065
   macro avg       0.86      0.52      0.51     77065
weighted avg       0.91      0.92      0.89     77065

TEST

              precision    recall  f1-score   support

         0.0       0.92      1.00      0.96     30357
         1.0       0.49      0.02      0.04      2671

    accuracy                           0.92     33028
   macro avg       0.71      0.51      0.50     33028
weighted avg       0.89      0.92      0.88     33028

CONFUSION MATRIX

col_0     0.0  1.0
TARGET            
0.0     30301   56
1.0      2617   54


In [128]:
print(roc_auc_score(y_trainn, lgb_model.predict_proba(X_trainn)[:,1]))
print(roc_auc_score(y_testt, lgb_model.predict_proba(X_testt)[:,1]))

0.8075124980723561
0.7267022001393205


In [129]:
lgb_model.fit(X_train, y_train)

LGBMClassifier(max_depth=3, n_estimators=500, random_state=21, reg_lambda=0.075)

In [130]:
TARGET = pd.DataFrame(lgb_model.predict_proba(X_test))
TARGET =TARGET.rename(columns={0:0, 1:'TARGET'}).drop(0, axis=1)
TARGET.head()

TARGET
0  0.054517
1  0.044096
2  0.058026
3  0.065720
4  0.002844

In [131]:
X_test__ = test.copy()
X_test__ = X_test__.sort_values(by = 'APPLICATION_NUMBER')
X_test__ = X_test__.reset_index(drop=True)
X_test__['TARGET'] = TARGET
X_test__ = X_test__.drop(['NAME_CONTRACT_TYPE'], axis=1)
X_test__.head()

APPLICATION_NUMBER    TARGET
0           123423342  0.054517
1           123423343  0.044096
2           123423344  0.058026
3           123423346  0.065720
4           123423347  0.002844

In [132]:
X_test_f = X_test_.copy()
X_test_f['TARGET_F'] = np.sqrt(X_test_['TARGET'] * X_test__['TARGET'])
#X_test_f['TARGET_F'] = X_test_['TARGET']*0.8 + X_test__['TARGET']*0.2
X_test_f.head()

APPLICATION_NUMBER    TARGET  TARGET_F
0           123423342  0.054287  0.054402
1           123423343  0.040208  0.042107
2           123423344  0.045050  0.051128
3           123423346  0.075531  0.070455
4           123423347  0.004340  0.003513

In [133]:
X_test_f = X_test_f.drop('TARGET',axis=1)
X_test_f = X_test_f.rename(columns={'TARGET_F':'TARGET'})
X_test_f.head()

APPLICATION_NUMBER    TARGET
0           123423342  0.054402
1           123423343  0.042107
2           123423344  0.051128
3           123423346  0.070455
4           123423347  0.003513

In [134]:
X_test_f.to_csv('test_submission.csv', index=False)

## Обзор данных - EDA

In [ ]:
X_train['TARGET'] = pd.DataFrame(y_train)

Категориальные переменные

In [ ]:
categorical_features = X_train.select_dtypes(include=['object'])
print(f"count of categorical_features {categorical_features.shape[1]}")

cat_features = categorical_features.columns.to_list()
cat_features

###### Анализ числовых переменных

In [ ]:
numerical_features = X_train.select_dtypes(exclude=['object'])
print(f"count of numeric_features {numerical_features.shape[1]}")

numerical_features.columns

###### Выявление дискретных признаков

In [ ]:
discrete_feature = [
    feature for feature in numerical_features
    if len(X_train[feature].unique())<25
]
print(f"Discrete Variables Count: {len(discrete_feature)}")

###### Непрерывные признаки

In [ ]:
continuous_feature = [
    feature for feature in numerical_features
    if feature not in discrete_feature
]

print(f"Continuous Feature Count {len(continuous_feature)}")

Для непрерывные признаков имеет смысл строить гистрограммы распределения или функции распределения, а также зависимость непрерывного признака в разрезе целевой переменной. Кроме того, имеет смысл строить распределение одновременно и для обучающей выборки, и для тестовой для понимания насколько отличаются распределения.

In [ ]:
# гистограммы распределения признаков:

fig, axes = plt.subplots(18, 2, figsize=(15, 60))

for num, feature in enumerate(continuous_feature):
    data = X_train[feature].copy()
    axes[num//2, num%2].hist(data, bins=200, color="blue", ec='white')
    axes[num//2, num%2].set_xlabel(feature, fontsize=10)

In [ ]:
fig, axes = plt.subplots(18, 2, figsize=(15, 60))

for num, feature in enumerate(continuous_feature):
    try:
        train_data, test_data = X_train[feature].copy(), X_test_[feature].copy()
        sns.kdeplot(train_data, ax=axes[num//2, num%2], color="blue", label="train")
        sns.kdeplot(test_data, ax=axes[num//2, num%2], color="green", label="test")
    except RuntimeError:
        pass
    except KeyError:
        train_data = train[feature].copy()
        sns.kdeplot(train_data, ax=axes[num//2, num%2], color="blue", label="train")
    axes[num//2, num%2].set_title(feature, fontsize=14)

### Корреляция цели с базовыми признаками¶

In [ ]:
correlation = numerical_features.corr()
corr_with_target = correlation["TARGET"].sort_values(ascending = False)
corr_with_target

In [ ]:
fig, axes = plt.subplots(figsize = (16,16))
plt.title("Correlation of Numeric Features with Sale Price", y=1, size=16)
sns.heatmap(correlation, square=True, vmax=0.8, cmap="viridis")

In [ ]:
TARGET_NAME = 'TARGET'
BASE_FEATURE_NAMES = X_train.columns.drop('TARGET').tolist()

corr_with_target = X_train[BASE_FEATURE_NAMES + [TARGET_NAME]].corr().iloc[:-1, -1].sort_values(ascending=False)

plt.figure(figsize=(10, 20))

sns.barplot(x=corr_with_target.values, y=corr_with_target.index)

plt.title('Correlation with target variable')
plt.show()

##### Произведём оценку признаков по методике WOE/IV

final_iv, IV = data_vars(X_train, y_train) # вычисление уровней IV (Information Values) метода WOE по фичам

IV.sort_values('IV') # вывод IV данных по фичам в отсортированном виде

##### Теперь проверим распределения выбранных фич в зависимости от целевой функции "0" или "1"¶

In [ ]:
fig, axes = plt.subplots(14, 3, figsize=(15, 60))
for num, feature in enumerate(numerical_features):
    try:
        feature_with_target_s = X_train[[feature, 'TARGET']].sample(1000)
        feature_s = feature_with_target_s[feature]
        feature_target_0 = feature_s[feature_with_target_s['TARGET'] == 0]
        feature_target_1 = feature_s[feature_with_target_s['TARGET'] == 1]
        sns.kdeplot(feature_target_0, ax=axes[num//3, num%3], color="blue", label="target=0")
        sns.kdeplot(feature_target_1, ax=axes[num//3, num%3], color="green", label="target=1")
    except RuntimeError:
        pass
    except KeyError:
        train_data = X_train[feature].copy()
        sns.kdeplot(feature_target_0, ax=axes[num//3, num%3], color="blue", label="target=0")
        sns.kdeplot(feature_target_1, ax=axes[num//3, num%3], color="green", label="target=1")
    axes[num//3, num%3].set_title(feature, fontsize=10)  

Отличная разрешающая способность

In [ ]:
for num, feature in enumerate(numerical_features):
    try:
        feature_with_target_s = X_train[[feature, 'TARGET']].sample(1000)
        feature_s = feature_with_target_s[feature]
        feature_target_0 = feature_s[feature_with_target_s['TARGET'] == 0]
        feature_target_1 = feature_s[feature_with_target_s['TARGET'] == 1]
        print(feature, mannwhitneyu(feature_target_0, feature_target_1))
    except RuntimeError:
        pass
    except KeyError:
        train_data = X_train[feature].copy()
        print(feature, mannwhitneyu(feature_target_0, feature_target_1))

In [ ]:
fig, axes = plt.subplots(12, 3, figsize=(15, 60))
for num, feature in enumerate(continuous_feature):
    try:
        feature_with_target_s = X_train[[feature, 'TARGET']].sample(10000)
        sns.pointplot(x=TARGET_NAME, y=feature, data=feature_with_target_s, ax=axes[num//3, num%3], capsize=.1)
    except RuntimeError:
        pass
    except KeyError:
        train_data = X_train[feature].copy()
        sns.pointplot(x='TARGET', y=feature, data=feature_with_target_s, ax=axes[num//3, num%3], capsize=.1)
    axes[num//3, num%3].set_title('Confidence intervals (95 %) for'+feature, fontsize=10)

##### Оценка дискретных признаков в разрезе целевой переменной

In [ ]:
features = ['CHILDRENS', 'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_YEAR']

fig, axes = plt.subplots(2, 2, figsize=(20, 20))
for num, feature in enumerate(features):
    try:
        sns.countplot(x=feature, hue='TARGET', data=X_train, ax=axes[num//2, num%2], capsize=.1)
    except RuntimeError:
        pass
    except KeyError:
        train_data = X_train[feature].copy()
        sns.countplot(x=feature, hue='TARGET', data=X_train_p, ax=axes[num//2, num%2], capsize=.1)
    axes[num//2, num%2].set_title(feature+' grouped by target variable', fontsize=10)

In [ ]:
X_train['ID'] = X_train.index.tolist()

In [ ]:
target_s = X_train.loc[X_train['CHILDRENS'].isin([0, 1, 2]), ['ID','CHILDRENS', TARGET_NAME]].sample(1000)
table = target_s.pivot_table(values='ID', index='CHILDRENS', columns=TARGET_NAME, aggfunc='count')
chi2, p, dof, expected = chi2_contingency(table, correction=False)
p

Т.о. можно объединить категории 0, 1 и 2

In [ ]:
target_s = X_train.loc[X_train['AMT_REQ_CREDIT_BUREAU_YEAR'].isin([1, 2, 3, 4, 5, 6, 7]), ['ID','AMT_REQ_CREDIT_BUREAU_YEAR', TARGET_NAME]].sample(1000)
table = target_s.pivot_table(values='ID', index='AMT_REQ_CREDIT_BUREAU_YEAR', columns=TARGET_NAME, aggfunc='count')
chi2, p, dof, expected = chi2_contingency(table, correction=False)
p

Т.о. можно объединить категории 1, 2, 3, 4, 5, 6, 7

#### Анализ пропусков

In [ ]:
msno.matrix(X_train)

In [ ]:
fig = plt.figure(figsize=(15, 5))
train_stats, test_stats = X_train.skew(), X_test.skew()
plt.plot(train_stats, color="blue", label="train")
plt.plot(test_stats, color="green", label="test")
plt.xticks(range(train_stats.shape[0]), train_stats.index, rotation=90)
plt.xlabel("features", size=15)
plt.xlim(0, len(train_stats))
plt.ylabel("skew", size=15)
plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 5))
train_stats, test_stats = X_train.kurtosis(), X_test.kurtosis()
plt.plot(train_stats, color="blue", label="train")
plt.plot(test_stats, color="green", label="test")
plt.xticks(range(train_stats.shape[0]), train_stats.index, rotation=90)
plt.xlabel("features", size=15)
plt.ylabel("kurtosis", size=15)
plt.xlim(0, len(train_stats))
plt.show()

In [ ]:
train_correlations = numerical_features.corr()
train_correlations = train_correlations.values.flatten()
train_correlations = train_correlations[train_correlations != 1]

columns = numerical_features.columns.tolist()
columns.remove("TARGET")

test_correlations = X_test[columns].corr()
test_correlations = test_correlations.values.flatten()
test_correlations = test_correlations[test_correlations != 1]

plt.figure(figsize=(20,5))
sns.distplot(train_correlations, color="Blue", label="train")
sns.distplot(test_correlations, color="Green", label="test")

plt.xlabel("Correlation values found in train (except 1)", size=14)
plt.title("Are there correlations between features?", size=14)
plt.legend(loc="best", fontsize=14)
plt.ylabel("Density", size=14)

трейн и тест одинаково показывают линейную корреляцию фичей к цели -> трейн и тест из одного сета,
модель точно будет стабильна в логистической регрессии